In [3]:
#imports
from tkinter import *
import os
import re
import time
import datetime

import telnetlib
import getpass 
import sys
import time
import serial
import threading #potentially replaced by multiprocessing
from multiprocessing.pool import ThreadPool

import pandas as pd
import numpy as np

from pdfrw import PdfReader
import matplotlib.pyplot as plt
import PyPDF2
from tkinter.filedialog import askopenfilename
os.environ.setdefault('MAGICK_HOME', '/usr/local/Cellar/imagemagick@6/6.9.9-34/')
from wand.image import Image
from PIL import Image as PI
import pyocr
import pyocr.builders
import io


#global defines
nport_ip = "10.136.1.99"
ref_port = "4001"


In [5]:

#updates the main status label with given text.  Could be accomplished by just calling "main_status.config" from within the main function...
def update_txt(next_instruction):
    main_status.config(text=next_instruction)   
    
#def connex(port):
    #print(port)
    
win = Tk()

win.title('SBE CTD Prequalification')
win.geometry('1000x600') # Size 200, 200

text = Text(win, height=2)

#labels
main_status = Label(win, text="Waiting for device selection")
calsheet_status_label = Label(win, text="")
sample_status_label = Label(win, text="")

#Buttons
port2 = Button(win, text="4002 - 485", command=lambda: instrument_connection("4002"))
port3 = Button(win, text="4003 - 232", command=lambda: instrument_connection("4003"))
port4 = Button(win, text="4004", command=lambda: instrument_connection("4004"))
#exit_button = Button(win, text="Exit", command=win.destroy())

text.insert(INSERT, "Please connect the RMA device and specify the port used")

text.pack()
port2.pack()
port3.pack()
port4.pack()
main_status.pack()
calsheet_status_label.pack()
sample_status_label.pack()

mainloop()

In [4]:
# Instrument Connection
#module to:
# 1. Prompt user to select NPORT port RMA instrument is connected to (maybe goes in main gui window?)
# 2. Create a telnet connection to both instruments
# 3. Send wakeup to both instruments
# 4. Alert user that connection successful

#entry point to module
def instrument_connection(rma_port):
    global nport_ip
    global ref_port
    device_dict = {
        "4002" : "id?\r\n",
        "4003" : "\r\n"
    }
    try:
        ref_telnet = telnetlib.Telnet(nport_ip, ref_port) #open connection
        rma_telnet = telnetlib.Telnet(nport_ip, rma_port)
        update_txt("Connecting, please wait...")
        time.sleep(1)
        ref_telnet.write(device_dict['4003'].encode("ascii"))  #wakeup both instruments
        rma_telnet.write((device_dict[rma_port]).encode("ascii"))
        time.sleep(1)
        rma_telnet.write((device_dict[rma_port]).encode("ascii"))
        time.sleep(1)
        rma_ok = rma_telnet.read_very_eager()
        ref_ok = ref_telnet.read_very_eager()
        ref_telnet.close() #close reference connection, not done with rma yet
        rma_telnet.close()
    except:
        print("bricked it")
        ref_telnet.close()
        rma_telnet.close()
    finally:
        ref_telnet.close()
        rma_telnet.close()
    update_txt("Connection Successful")
    calsheet_status_label.config(text="Collecting RMA calibration data...")    
    compcals = compare_cals_entrypoint(rma_port) #pass the telnet connection to compare_cals function
    gen_report(compcals)    
    
    
# Compare Cals
#module to:
# 1. Prompt user for calibration sheet location
# 2. Collect cals from instrument(dcal, #iidc)
# 3. Compare Results
# 4. Display side by side results AND suggest an action(pass/fail)
# 5. If "pass" then continue after ~30seconds if "fail" pause and wait for user?\
def conv_to_float(x):  #lambda float conversion
    try:
        return float(x)
    except:
        return 0

def compare_cals_entrypoint(rma_port):
    instr_test = 'SBE37SI-RS485 V 3.2  5679\r\ntemperature:  25-Mar-17\r\n    TA0 = 4.261456e-05\r\n    TA1 = 2.731360e-04\r\n    TA2 = -2.206636e-06\r\n    TA3 = 1.490857e-07\r\nconductivity:  25-Mar-17\r\n    G = -9.994807e-01\r\n    H = 1.566288e-01\r\n    I = -3.317297e-04\r\n    J = 4.905268e-05\r\n    CPCOR = -9.570000e-08\r\n    CTCOR = 3.250000e-06\r\n    WBOTC = 1.185617e-06\r\nreference pressure = 2007.0 decibars\r\n<Executed/>\r\n'
    sheet_test = open("Output.txt", 'r').read()
    if rma_port == "4002":
        ins_caldata = get_cals_37()
    if rma_port == "4003":
        ins_caldata = get_cals_19()
    sheet_caldata = get_sheet_cals()
    cal_values = compare_cals(ins_caldata, sheet_caldata)
    #update_cals(df)  ##THOROUGH TEST BEFORE FIRING THIS ONE
    return cal_values
    
def get_cals_37():
    global nport_ip
    rma_telnet = telnetlib.Telnet(nport_ip, "4002")
    rma_telnet.write("id?\r\n".encode("ascii"))
    time.sleep(1)
    rma_telnet.write("id?\r\n".encode("ascii"))
    time.sleep(1)
    sbe_id = str(rma_telnet.read_very_eager())[5:7]
    rma_telnet.write(("#"+sbe_id+"DC\r\n").encode("ascii"))
    time.sleep(1)
    caldata = rma_telnet.read_very_eager()
    rma_telnet.close()
    return caldata #.decode('unicode-escape')
def get_cals_19():
    rma_telnet = telnetlib.Telnet(nport_ip, "4002")
    rma_telnet.write("DCal\r\n".encode("ascii"))
    time.sleep(1)
    rma_telnet.write("DCal\r\n".encode("ascii"))
    time.sleep(1)
    caldata = rma_telnet.read_very_eager()
    rma_telnet.close()
    return str(caldata)

#this function doesn't actually need to exist, but helps with clarity i think?
def get_sheet_cals():
    filepath = request_filepath()
    return read_file(filepath)
    

#get user input filepath(browse files)    
def request_filepath():
    file_select = Tk()
    file_select.withdraw()
    filename=askopenfilename()
    file_select.update()
    file_select.destroy()
    return filename

#OCR on file, parse out to text
def read_file(filepath):
    tool = pyocr.get_available_tools()[0]
    lang = tool.get_available_languages()[0]
    req_image = []
    final_text = []
    image_pdf = Image(filename=filepath, resolution=300)
    image_jpeg = image_pdf.convert('jpeg')
    for img in image_jpeg.sequence:
        img_page = Image(image=img)
        req_image.append(img_page.make_blob('jpeg'))
    for img in req_image: 
        txt = tool.image_to_string(
            PI.open(io.BytesIO(img)),
            lang=lang,
            builder=pyocr.builders.TextBuilder()
        )
        final_text.append(txt)
    merge_text = ''.join(final_text)
    return merge_text

def split_sheet(sheet):
    try:
        m = sheet[0].split(' ')[0]
        return m
    except:
        pass

def compare_cals(instr, sheet):
    i_s = instr
    s_s = sheet
    comparison_array = []
    instrument = parse_cal_tags(i_s.decode('unicode-escape'))
    sheet_return = parse_sheets(s_s, instrument)     
    sheet_return = clean_sheet_data(sheet_return)
    return (instrument, sheet_return)
    
def find_value(tag, input_file):
    m = re.findall(tag, input_file)
    if m:
        return m
    else:
        return "0"

def split_sheet(sheet):
    try:
        m = sheet[0].split(' ')[0]
        return m
    except:
        pass

def parse_sheets(s_s, instr):    
    sheet_return = []
    instrument = instr
    for i in range(len(instrument[0])):
        tags = instrument[0]
        sheet_regex = r"(?i)(?<=\s" + instrument[0][i] + " = )(.*)(?=\s)"
        m = re.findall(sheet_regex, s_s)
        sheet_return.append(split_sheet(m))
    return sheet_return
    
                   
def parse_cal_tags(cal_string):
    m = re.findall(r"(?<=\s)(.*)(?=\s)", cal_string) 
    calibration_tags = []
    calibration_rows = []
    for x in m:
        entry = x.split('=')
        try:
            if len(entry) >= 2:
                calibration_tags.append(entry[0].strip())
                calibration_rows.append(entry[1].strip())
        except:
            pass
    return (calibration_tags, calibration_rows)

def clean_sheet_data(sheet_data):
    cleanup_dict = {"~":"-",
                   "€":"e",
                   "|":"1",
                   "—":"-",
                   "l":"1",
                   "O":"0",
                   "0":"0"
                   }
    cleanup_list = ["~",                    
                    "€",
                   "|",
                   "—",
                   "l",
                   "O",
                   "0"]
    cleaned_list=[]
    for i in sheet_data:
        clean = i
        if clean:
            for i in range(len(cleanup_list)):
                clean = clean.replace(cleanup_list[i], cleanup_dict[cleanup_list[i]])
        cleaned_list.append(clean)
    return cleaned_list

##Helper function to clear whatever is on the frame currently  
def clear_screen():
    for widget in win.winfo_children():
        widget.destroy()

#### Generates the Calibration Report with option to update values
###
##
#
def gen_report(cals):
    def stringvar_list(lst):
        vals = [str(i) for i in lst]
        ret_lst = [None]*len(lst)
        for i in range(len(lst)):
            ret_lst[i] = StringVar()
            ret_lst[i].set(vals[i])
        return ret_lst
    def update_cal_var():
        for i in range(length):
            try:
                sheet_values[i].set(update_val[i].get())
            except Exception as e:
                print (e)

    clear_screen()
                
    instr_data = cals[0]
    sheet_data = cals[1]
       
    sheet_data = ['None' if v is None else v for v in sheet_data]

    instr_tags = stringvar_list(instr_data[0])
    instr_values = stringvar_list(instr_data[1])
    sheet_values = stringvar_list(sheet_data)
    
    length = len(instr_data[0])
    tags = [None] * length
    ins_val = [None] * length
    she_val = [None] * length
    update_entry = [None] * length
    update_val = [None]*length
    
    cal_label = Label(win, text="Calibration Comparison", height=1, width=20)
    cal_label.grid(sticky='NW', columnspan=3)
          
    for i in range(length):
        tags[i] = Text(win, height=1, width=15)
        tags[i].insert(INSERT, instr_tags[i].get())
        tags[i].grid(column=0, row=i+1)
        ins_val[i] = Text(win, height=1, width=15)
        ins_val[i].insert(INSERT, instr_values[i].get())
        ins_val[i].grid(column=1, row=i+1, sticky='NW')
        she_val[i] = Label(win, height=1, width=15, text=sheet_values[i].get() , textvariable=sheet_values[i])
        she_val[i].grid(column=2, row=i+1, sticky='NW', padx = 10)
        update_val[i] = StringVar()
        update_entry[i] = Entry(win ,textvariable = update_val[i])
        update_entry[i].insert(END, sheet_values[i].get())
        update_entry[i].grid(column=3, row=i+1, sticky='NW')
    
    update_button = Button(win, text='Update Values', command=update_cal_var)
    update_button.grid(row=length+1, columnspan=3)
        

In [8]:
#
    

[None,
 None,
 None,
 None,
 '-1.017709e+000',
 '1.328397e-001',
 '-1.413619e-004',
 None,
 '-9.5700e-008',
 '3.2500e-006',
 None,
 None]

In [ ]:
    #df = pd.DataFrame(data = compare_cals(ins_caldata, sheet_caldata), columns=['Tag', 'Ins Val', 'Sheet Val'])#columns=['Value', 'Instrument', 'Sheet'])
#     try:
#         df['Ins Val'] = df['Ins Val'].apply(lambda x: x.replace('~','-').replace('€', 'e'))
#         df['Sheet Val'] = df['Sheet Val'].apply(lambda x: x.replace('~','-').replace('€', 'e'))
#     except: 
#         pass
    #df['Instrument'] = df['Instrument'].apply(lambda x: conv_to_float(x))
    #df['Sheet'] = df['Sheet'].apply(lambda x: conv_to_float(x))   
    #df['Difference'] = abs((df['Instrument']) - (df['Sheet']))

In [5]:
dit = {"hi":"ho"}
print(dit["hi"])

ho
